In [1]:
import time

from pprint import pprint
from tqdm import tqdm
from neo4j import GraphDatabase


In [2]:
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, parameters=None, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query, parameters))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response


In [3]:
conn = Neo4jConnection(uri="bolt://neo4j:7687", user="neo4j", pwd="uom-pkg")


In [4]:
conn.query('CREATE CONSTRAINT q_value IF NOT EXISTS ON (n:Node) ASSERT n.id IS UNIQUE')

[]

In [5]:
cqlNodeQuery          = "MATCH (x:university) RETURN x"

cqlCreate = """CREATE (cornell:university { name: "Cornell University"}),
(yale:university { name: "Yale University"}),
(princeton:university { name: "Princeton University"}),
(harvard:university { name: "Harvard University"}),

(cornell)-[:connects_in {miles: 259}]->(yale),
(cornell)-[:connects_in {miles: 210}]->(princeton),
(cornell)-[:connects_in {miles: 327}]->(harvard),

(yale)-[:connects_in {miles: 259}]->(cornell),
(yale)-[:connects_in {miles: 133}]->(princeton),
(yale)-[:connects_in {miles: 133}]->(harvard),

(harvard)-[:connects_in {miles: 327}]->(cornell),
(harvard)-[:connects_in {miles: 133}]->(yale),
(harvard)-[:connects_in {miles: 260}]->(princeton),

(princeton)-[:connects_in {miles: 210}]->(cornell),
(princeton)-[:connects_in {miles: 133}]->(yale),
(princeton)-[:connects_in {miles: 260}]->(harvard)"""

conn.query(cqlCreate)
nodes = conn.query(cqlNodeQuery)
for node in nodes:
    print(node)


<Record x=<Node id=0 labels=frozenset({'university'}) properties={'name': 'Cornell University'}>>
<Record x=<Node id=1 labels=frozenset({'university'}) properties={'name': 'Yale University'}>>
<Record x=<Node id=2 labels=frozenset({'university'}) properties={'name': 'Princeton University'}>>
<Record x=<Node id=3 labels=frozenset({'university'}) properties={'name': 'Harvard University'}>>


In [18]:
cqlCreate2 = """
CREATE (VariableAssignment:KnowledgeUnit {externalId: 1,
name: "VariableAssignment",
description: "You can assign a value to a variable to use that value in different operations.", 
code_example: "x = 23
size = 120
name = ’Joe’", 
exercise: "Create a variable, name it to max_speed and assign 130 to it."}),

(Literals:KnowledgeUnit { externalId: 2,
name: "Literals",
description: "Quantities/ notations whose value does not change during the execution of a program. Also known as constants. 
String Literals
Numeric Literals
Boolean Literals
Literal Collections
Special Literals", 
code_example: "423
-87
True", 
exercise: "Use numeric literals to calculate 5 multiplied by 8."}),

(VariableAssignment)-[:uses]->(Literals)

"""
conn.query(cqlCreate2)


[]

In [9]:
cqlCreate3 = """
CREATE (Introduction:KnowledgeUnit { externalId: 1,
name: "Introduction",
description: "It needs to be started somewhere: Python is a programming language. It is named after the BBC show Monty Python’s Flying Circus",
timeInstances: ["3.8", "3.9"]}),

(Literal:KnowledgeUnit { externalId: 2,
name: "Literal",
description: "Quantities/ notations whose value does not change during the execution of a program. Also known as constants. 
String Literals
Numeric Literals
Boolean Literals
Literal Collections
Special Literals",
timeInstances: ["*"]}),

(WriteToConsole:KnowledgeUnit { externalId: 3,
name: "WriteToConsole",
description: "Print the specified message to the screen, or other standard output device. The message can be a string, or any other object, the object will be converted into a string before written to the screen.",
timeInstances: ["*"],
evokers: ["print"]}),

(DetermineObjectType:KnowledgeUnit { externalId: 4,
name: "DetermineObjectType",
description: "The type of a Python object determines what kind of object it is; every object has a type. An object’s type is accessible as its __class__ attribute or can be retrieved with type(obj).",
timeInstances: ["*"],
evokers: ["type"]}),

(ListDataType:KnowledgeUnit { externalId: 5,
name: "ListDataType",
description: "Lists are iterable objects. How to set a list element, how to delete a list element and how to get a list element.",
timeInstances: ["*"],
evokers: ["[", "]"]}),

(ForLoop:KnowledgeUnit { externalId: 6,
name: "ForLoop",
description: "It iterates over an iterable object, capturing each element to a local variable for use by the attached block.",
timeInstances: ["*"],
evokers: ["for", "in"]}),

(CodeBlock:KnowledgeUnit { externalId: 7,
name: "CodeBlock",
description: "Program code that belongs together. Python uses whitespace indentation, rather than curly brackets or keywords, to delimit blocks. An increase in indentation comes after certain statements; a decrease in indentation signifies the end of the current block. Thus, the program's visual structure accurately represents its semantic structure.",
timeInstances: ["*"],
evokers: [""]}),

(AssignmentStatement:KnowledgeUnit { externalId: 8,
name: "AssignmentStatement",
description: "The assignment statement (=) binds a name as a reference to a separate, dynamically allocated object. Variables may subsequently be rebound at any time to any object.",
timeInstances: ["*"],
evokers: ["="]}),

(Statement:KnowledgeUnit { externalId: 9,
name: "Statement",
description: "In computer programming, a statement is a syntactic unit of an imperative programming language that expresses some action to be carried out. A program written in such a language is formed by a sequence of one or more statements. A statement may have internal components (e.g. expressions).",
timeInstances: ["*"],
evokers: [""]}),

(StringLiteral:KnowledgeUnit { externalId: 10,
name: "StringLiteral",
description: "A string literal or anonymous string is a literal for a string value in the source code of a computer program. Modern programming languages commonly use a quoted sequence of characters, formally 'bracketed delimiters', as in x = 'foo', where 'foo' is a string literal with value foo. ",
timeInstances: ["*"],
evokers: [""]}),

(StringDataType:KnowledgeUnit { externalId: 11,
name: "StringDataType",
description: "String variables - which are also called alphanumeric variables or character variables -- have values that are treated as text.",
timeInstances: ["*"],
evokers: [""]}),

(Variable:KnowledgeUnit { externalId: 12,
name: "Variable",
description: "Value storage",
timeInstances: ["*"],
evokers: "="}),

(WhileLoop:KnowledgeUnit { externalId: 13,
name: "WhileLoop",
description: "Executes a block of code as long as its condition is true.",
timeInstances: ["*"],
evokers: "while"}),

(Condition:KnowledgeUnit { externalId: 14,
name: "Condition",
description: "Conditionally executes a block of code.",
timeInstances: ["*"],
evokers: "if"}),

(VariableNaming:KnowledgeUnit { externalId: 21,
name: "VariableNaming",
description: "Use  long speaking names for variables. Shorter temporary names can be used in longer formulas to achieve clean and understandable code at the same time.",
timeInstances: ["*"],
evokers: "="}),


(VariableAssignment:KnowledgeUnit { externalId: 22,
name: "VariableAssignment",
description: "You can assign a value to a variable to use that value in different operations.",
timeInstances: ["3.8", "3.9"],
evokers: "="}),

(Algorithm:KnowledgeUnit { externalId: 23,
name: "Algorithm",
description: "A list set of instructions, used to solve problems or perform tasks, based on the understanding of available alternatives.",
timeInstances: ["*"],
evokers: [""]}),

(ModifyACollectionWithCopy:KnowledgeUnit { externalId: 24,
name: "ModifyACollectionWithCopy",
description: " Strategy1:  Iterate over a copy; Strategy2:  Create a new collection",
timeInstances: ["*"],
evokers: [""]}),


(Literal)-[:prerequisite_of]->(VariableAssignment),
(AssignmentStatement)-[:prerequisite_of]->(VariableAssignment),
(VariableAssignment)-[:prerequisite_of]->(ListDataType),
(ListDataType)-[:prerequisite_of]->(ForLoop),
(VariableAssignment)-[:prerequisite_of]->(ForLoop),
(CodeBlock)-[:prerequisite_of]->(ForLoop),
(CodeBlock)-[:prerequisite_of]->(WhileLoop),
(Condition)-[:prerequisite_of]->(WhileLoop),
(CodeBlock)-[:prerequisite_of]->(Condition),
(StringLiteral)-[:prerequisite_of]->(StringDataType),
(StringDataType)-[:prerequisite_of]->(WriteToConsole),
(VariableAssignment)-[:prerequisite_of]->(DetermineObjectType),
(Variable)-[:prerequisite_of]->(VariableNaming),
(VariableAssignment)-[:prerequisite_of]->(VariableNaming),

(ForLoop)-[:prerequisite_of]->(ModifyACollectionWithCopy),
(Condition)-[:prerequisite_of]->(ModifyACollectionWithCopy),
(ListDataType)-[:prerequisite_of]->(ModifyACollectionWithCopy),

(ForLoop)-[:is_a]->(Statement),
(WhileLoop)-[:is_a]->(Statement),
(AssignmentStatement)-[:is_a]->(Statement),
(StringLiteral)-[:is_a]->(Literal),
(ModifyACollectionWithCopy)-[:is_a]->(Algorithm),

(Test456:TestUnit { externalId: 456,
description: "One type of variable assignment test.",
url: "https://localhost/uom-python-test/",
timeInstances: ["3.9"]}),

(Test678:TestUnit { externalId: 678,
description: "Another type of test",
url: "https://localhost/uom-python-test/",
timeInstances: ["3.8"]}),

(Test456)-[:tests]->(VariableAssignment),
(Test678)-[:tests]->(VariableAssignment),

(Material123:MaterialUnit { externalId: 123,
description: "Some introduction to version 3.8.*",
url: "https://localhost/uom-python-material/",
timeInstances: ["3.8"]}),

(Material234:MaterialUnit { externalId: 234,
description: "Intro text to version 3.9.*",
url: "https://localhost/uom-python-material/",
timeInstances: ["3.9"]}),

(Material235:MaterialUnit { externalId: 235,
description: "Intro text to version 3.10.*",
url: "https://localhost/uom-python-material/",
timeInstances: ["3.10"]}),

(Material123)-[:material_of]->(Introduction),
(Material234)-[:material_of]->(Introduction),
(Material235)-[:material_of]->(Introduction)




"""
conn.query(cqlCreate3)


[]

In [6]:
fruits = ["Apple", "Banana", "Lemon", "Grapes" ]
for fruit in fruits:
    print(fruit)

Apple
Banana
Lemon
Grapes
